## LangChain - Prompts

Typical Prompt for RAG
- Rules for out LLM (System prompt)
- Context (Knowledge source)
- Question (User prompt)
- Answer

### Basic Prompting

In [1]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [2]:
prompt = """"
    Answer the user's query based on the context below. 
    If you cannot answer the question using the 
    provided information answer with "I dont know".

    Context: {context}
"""

In [3]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("user", "{query}")
])

In [4]:
prompt_template.input_variables

['context', 'query']

In [5]:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='"\n    Answer the user\'s query based on the context below. \n    If you cannot answer the question using the \n    provided information answer with "I dont know".\n\n    Context: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

In [ ]:
# More explicit way to define the role of the message
prompt_template_2 = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prompt),
    HumanMessagePromptTemplate.from_template("{query}")
])

In [7]:
# Note that is the exactly same as prompt_template.messages
prompt_template_2.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='"\n    Answer the user\'s query based on the context below. \n    If you cannot answer the question using the \n    provided information answer with "I dont know".\n\n    Context: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

### Invoking LLM with Templates

In [8]:
from langchain_openai import ChatOpenAI

In [18]:
model = "gpt-4o-mini"

In [19]:
llm = ChatOpenAI(temperature=0.0, model=model)

In [21]:
context = """
    Aurelio AI is an AI company developing tooling for AI
    engineers. Their focus is on language AI with the team having strong
    expertise in building AI agents and a strong background in
    information retrieval.

    The company is behind several open source frameworks, most notably
    Semantic Router and Semantic Chunkers. They also have an AI
    Platform providing engineers with tooling to help them build with
    AI. Finally, the team also provides development services to other
    organizations to help them bring their AI tech to market.

    Aurelio AI became LangChain Experts in September 2024 after a long
    track record of delivering AI solutions built with the LangChain
    ecosystem.
"""

query = "What does Aurelio AI do?"

In [25]:
# NOTE: We're using LangChain Expression Language (LCEL)
# the '|' (aka pipe) connect our each component
pipeline = prompt_template | llm

In [24]:
answer = pipeline.invoke({"query": query, "context": context})
answer.content

'Aurelio AI is an AI company that develops tooling for AI engineers, focusing on language AI. They have expertise in building AI agents and information retrieval. The company is known for several open source frameworks, including Semantic Router and Semantic Chunkers, and offers an AI Platform that provides engineers with tools to build with AI. Additionally, they provide development services to help other organizations bring their AI technology to market.'

In [ ]:
# NOTE: It works without the dict
pipeline_2 = (
    {
        "query": lambda x: x["query"],
        "context": lambda x: x["context"]
    }
    | prompt_template_2
    | llm
)

In [28]:
answer_2 = pipeline_2.invoke({"query": query, "context": context})
answer_2.content

'Aurelio AI is an AI company that develops tooling for AI engineers, focusing on language AI. They have expertise in building AI agents and information retrieval. The company is known for several open source frameworks, including Semantic Router and Semantic Chunkers, and offers an AI Platform that provides engineers with tools to build with AI. Additionally, they provide development services to help other organizations bring their AI technology to market.'

### Few-shot prompting

In [31]:
from langchain.prompts import FewShotChatMessagePromptTemplate

In [29]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

In [30]:
examples = [
    {"input": "Here is query #1", "output": "Here is answer #1"},
    {"input": "Here is query #2", "output": "Here is answer #2"},
    {"input": "Here is query #3", "output": "Here is answer #3"}
]

In [33]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)
print(few_shot_prompt.format())

Human: Here is query #1
AI: Here is answer #1
Human: Here is query #2
AI: Here is answer #2
Human: Here is query #3
AI: Here is answer #3


In [34]:
new_system_prompt = """
    Answer the user's query based on the context below.
    If you cannot answer the question using the
    provided information answer with "I don't know".

    Always answer in markdown format. When doing so please
    provide headers, short summaries, follow with bullet
    points, then conclude.

    Context: {context}
""" 

In [35]:
prompt_template.messages[0].prompt.template = new_system_prompt

In [36]:
out = pipeline.invoke({"query": query, "context": context}).content
print(out)

# Overview of Aurelio AI

Aurelio AI is an AI company that specializes in developing tools and services for AI engineers, particularly in the realm of language AI. 

## Key Focus Areas
- **Language AI Development**: Expertise in building AI agents and information retrieval systems.
- **Open Source Frameworks**: Creator of notable frameworks like Semantic Router and Semantic Chunkers.
- **AI Platform**: Provides tooling for engineers to facilitate AI development.
- **Development Services**: Offers assistance to organizations in bringing their AI technologies to market.

## Achievements
- Became LangChain Experts in September 2024, showcasing their proficiency in the LangChain ecosystem.

In conclusion, Aurelio AI is dedicated to empowering AI engineers through innovative tools, frameworks, and development services in the language AI space.


In [37]:
examples = [
    {
        "input": "Can you explain gravity?",
        "output": (
            "## Gravity\n\n"
            "Gravity is one of the fundamental forces in the universe.\n\n"
            "### Discovery\n\n"
            "* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n"
            "* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n"
            "### In General Relativity\n\n"
            "* Gravity is described as the curvature of spacetime.\n"
            "* The more massive an object is, the more it curves spacetime.\n"
            "* This curvature is what causes objects to fall towards each other.\n\n"
            "### Gravitons\n\n"
            "* Gravitons are hypothetical particles that mediate the force of gravity.\n"
            "* They have not yet been detected.\n\n"
            "**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n"
        )
    },
    {
        "input": "What is the capital of France?",
        "output": (
            "## France\n\n"
            "The capital of France is Paris.\n\n"
            "### Origins\n\n"
            "* The name Paris comes from the Latin word \"Parisini\" which referred to a Celtic people living in the area.\n"
            "* The Romans named the city Lutetia, which means \"the place where the river turns\".\n"
            "* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.\n\n"
            "**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.\n\n"
        )
    }
]

In [39]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)
few_shot_prompt

FewShotChatMessagePromptTemplate(examples=[{'input': 'Can you explain gravity?', 'output': '## Gravity\n\nGravity is one of the fundamental forces in the universe.\n\n### Discovery\n\n* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n### In General Relativity\n\n* Gravity is described as the curvature of spacetime.\n* The more massive an object is, the more it curves spacetime.\n* This curvature is what causes objects to fall towards each other.\n\n### Gravitons\n\n* Gravitons are hypothetical particles that mediate the force of gravity.\n* They have not yet been detected.\n\n**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n'}, {'input': 'What is the capital of France?', 'output': '## France\n\nThe capital of France is Paris.\n\n### Origins\n\n* The name Paris comes from the Latin word "Parisini" which refe

In [40]:
prompt_template_3 = ChatPromptTemplate.from_messages([
    ("system", new_system_prompt),
    few_shot_prompt,
    ("user", "{query}")
])

In [41]:
pipeline_3 = prompt_template_3 | llm

In [42]:
answer_3 = pipeline_3.invoke({"query": query, "context": context})

In [43]:
print(answer_3.content)

## Aurelio AI Overview

Aurelio AI is an AI company focused on developing tools and services for AI engineers, particularly in the realm of language AI.

### Key Areas of Focus

- **Language AI**: Specializes in creating AI solutions that understand and generate human language.
- **AI Agents**: Expertise in building intelligent agents that can perform tasks autonomously.
- **Information Retrieval**: Strong background in retrieving and processing information effectively.

### Products and Services

- **Open Source Frameworks**: 
  - **Semantic Router**: A framework for managing semantic tasks.
  - **Semantic Chunkers**: A tool for breaking down and processing language data.
  
- **AI Platform**: 
  - Provides engineers with tools to facilitate AI development.
  
- **Development Services**: 
  - Offers assistance to organizations in bringing their AI technologies to market.

### Recognition

- Became **LangChain Experts** in September 2024, showcasing their proficiency in the LangChain e

### Chain of Thought prompting

In [44]:
no_cot_system_prompt = """
    Be a helpful assistant and answer the user's question.
    You MUST answer the question directly without any other
    text or explanation.
"""

In [45]:
no_cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", no_cot_system_prompt),
    ("user", "{query}")
])

In [50]:
query = (
    "How many keystrokes are needed to type the numbers 1 to 500?"
)

In [51]:
no_cot_pipeline = no_cot_prompt_template | llm

In [52]:
no_cot_result = no_cot_pipeline.invoke({"query": query})

In [53]:
print(no_cot_result.content)

The total number of keystrokes needed to type the numbers 1 to 500 is 1,511.


In [55]:
cot_system_prompt = """
    Be a helpful assistant and answer the user's question. 

    To answer que question you must:
    - List systematically and in precise detail all subproblem
    that need to be solved to answer the question.
    - Solve each sub problem INDIVIDUALLY and in sequence.
    - Finally, use everything you have worked through to 
    provide the final answer. 
"""

In [56]:
cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", cot_system_prompt),
    ("user", "{query}")
])

In [57]:
cot_pipeline = cot_prompt_template | llm

In [58]:
cot_result = cot_pipeline.invoke({"query": query})

In [59]:
print(cot_result.content)

To determine how many keystrokes are needed to type the numbers from 1 to 500, we can break this problem down into several subproblems:

1. **Count the numbers in each range**:
   - Count the numbers from 1 to 9 (1-digit numbers).
   - Count the numbers from 10 to 99 (2-digit numbers).
   - Count the numbers from 100 to 499 (3-digit numbers).
   - Count the number 500 (a 3-digit number).

2. **Calculate the total number of keystrokes for each range**:
   - Calculate the total keystrokes for 1-digit numbers.
   - Calculate the total keystrokes for 2-digit numbers.
   - Calculate the total keystrokes for 3-digit numbers.

3. **Sum the total keystrokes from all ranges**.

Now, let's solve each subproblem step by step.

### Subproblem 1: Count the numbers in each range

1. **1-digit numbers (1 to 9)**:
   - There are 9 numbers (1, 2, 3, 4, 5, 6, 7, 8, 9).

2. **2-digit numbers (10 to 99)**:
   - There are 90 numbers (from 10 to 99 inclusive).

3. **3-digit numbers (100 to 499)**:
   - Ther